# Data Privacy Final Project
### Bradon Soucy and Sam Will

In [ ]:
import pandas as pd
import numpy as np
import csv
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

spotify = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
spotify = spotify.dropna()

## TODO:
- change artist map to genre map
- then alter the below code such genre map is the new y
- Note this will mean you have to alter which columns you drop and normalize look at code below Data Notes

In [ ]:
# s.head()

In [ ]:

genre_map = {}
for r in spotify['playlist_genre']:
    if r not in genre_map.keys():
        genre_map[r] = len(genre_map)
print(genre_map)   

# Data Notes
- spotify = mother data
- spotify_X = Important data (- track name) (Test w/ genre-subgenre-both)
- spotify_ Y = track name

In [ ]:
# drop unnecessary columns for ML in X_data (i.e. cols with a lot of unique text)
spotify_X = spotify.copy()
to_drop = ['track_id','track_name', 'track_artist', 'track_album_id', 'track_album_name', 'track_album_release_date', 'playlist_name', 
          'playlist_id',  'playlist_subgenre']
spotify_X.drop(to_drop, inplace=True, axis=1)

# Y data:
spotify_X['playlist_genre'] = spotify_X['playlist_genre'].map(genre_map)
spotify_y = spotify_X['playlist_genre']
spotify_X.drop('playlist_genre', inplace=True, axis=1)

# Normalize data in an attempt to improve accuracy:
# Duration_ms -> Duration_min
dur_min = spotify_X['duration_ms'].apply(lambda x: x / 600000)
spotify_X['duration_ms'] = dur_min
spotify_X.rename(columns={'duration_ms': 'duration_min'}, inplace=True)

# tempo: /100
tempo_alt = spotify_X['tempo'].apply(lambda x: x / 1000)
spotify_X['tempo'] = tempo_alt
spotify_X.rename(columns={'tempo': 'tempo_1'}, inplace=True)

# Popularity rank /100
t_pop_alt = spotify_X['track_popularity'].apply(lambda x: x / 100)
spotify_X['track_popularity'] = t_pop_alt
spotify_X.rename(columns={'track_popularity': 'track_popularity_1'}, inplace=True)

# loudness col
loudness_plus = spotify_X['loudness'].apply(lambda x: (x + .175130806040521) / -10)
spotify_X['loudness'] = loudness_plus
spotify_X.rename(columns={'loudness': 'loudness_1'}, inplace=True)

# Display new DF
print(spotify_X.head())
print("---------------------------------------------------------")
print(spotify_y)

# Notes

- Input Features (X_train): X_train represents the input features or independent variables of your dataset.
    - Each row of X_train corresponds to a data point, and each column corresponds to a specific feature or attribute of that data point.
- Target Labels (y_train): y_train represents the target labels or dependent variables of your dataset.
    - y_train would contain the class labels (spam or not spam) corresponding to each data point in X_train.
- In summary, X_train holds the input data, and y_train holds the corresponding labels that you want your machine learning model to learn and predict. 

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(spotify_X, spotify_y, test_size=0.2, random_state=42)

print('Train and test set sizes:', len(X_train), len(X_test))
print('Train and test set sizes:', len(y_train), len(y_test))

In [ ]:
def train_model():
    model = LogisticRegression().fit(X_train, y_train)
    return model

model = train_model()
print('Model coefficients:', model.coef_[0])
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])